# Demo de Análisis de Colores con Snowflake AI SQL

**Autor:** [Tu Nombre]

**Objetivo:**

Analizar y comparar la paleta de colores de Ipesa Pinturas contra las tendencias actuales de diseño de interiores y los catálogos de Comex y Sherwin-Williams, utilizando Snowflake y AI SQL. El objetivo es identificar los colores de Ipesa con mayor potencial de ventas alineados a las tendencias del mercado.

---

## Índice
1. [Configuración y Recolección de Datos](#configuracion)
2. [Estructuración y Carga en Snowflake](#estructura)
3. [Análisis con AI SQL](#analisis)
4. [Visualización de Resultados](#visualizacion)
5. [Conclusiones](#conclusiones)



## 1. Configuración y Recolección de Datos <a name="configuracion"></a>

En esta sección se realiza el scraping de tendencias de color y catálogos de pintura de las marcas objetivo.

- **Tendencias de color:** Pantone, Architectural Digest, Behance, Domino.
- **Catálogos de pintura:** Ipesa, Comex, Sherwin-Williams.

Se extraen: nombre del color, descripción, código HEX/RGB y fuente.



In [ ]:
# Librerías necesarias para scraping y manejo de datos
!pip install requests beautifulsoup4 pandas snowflake-connector-python plotly matplotlib

import requests
from bs4 import BeautifulSoup
import pandas as pd
import uuid



### Ejemplo: Scraping de tendencias de color (Pantone)

> **Nota:** Adapta los selectores según la estructura real de cada sitio. Repite el proceso para los otros sitios de tendencias y catálogos de pintura.



### Ejemplo: Scraping de catálogo de pintura (Ipesa)

> **Nota:** Repite el proceso para Comex y Sherwin-Williams.



In [ ]:
# Ejemplo de scraping de catálogo (Ipesa)
url = "https://www.ipesamex.com/catalogo-colores"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

colores = []
for color in soup.select('.color-card'):
    nombre = color.select_one('.color-name').text.strip()
    codigo = color.select_one('.color-code').text.strip() if color.select_one('.color-code') else None
    hex_color = color.select_one('.color-sample')['style'].split('#')[-1][:6]
    colores.append({
        'marca': 'Ipesa',
        'nombre_color': nombre,
        'codigo_color': codigo,
        'hex_color': f"#{hex_color}"
    })

df_ipesa = pd.DataFrame(colores)
df_ipesa.head()



## 2. Estructuración y Carga en Snowflake <a name="estructura"></a>

En esta sección se define el esquema de tablas y se cargan los datos recolectados a Snowflake.



### DDL: Definición de Tablas en Snowflake

```sql
CREATE TABLE tendencias_colores (
    id_tendencia VARCHAR PRIMARY KEY,
    nombre_tendencia VARCHAR,
    descripcion_tendencia VARCHAR,
    hex_color VARCHAR,
    fuente_sitio VARCHAR
);

CREATE TABLE catalogo_pinturas (
    id_color VARCHAR PRIMARY KEY,
    marca VARCHAR,
    nombre_color VARCHAR,
    hex_color VARCHAR
);
```



In [ ]:
# Conexión y carga de datos a Snowflake
import snowflake.connector

conn = snowflake.connector.connect(
    user='TU_USUARIO',
    password='TU_PASSWORD',
    account='TU_CUENTA',
    warehouse='TU_WAREHOUSE',
    database='TU_DATABASE',
    schema='PUBLIC'
)

# Insertar tendencias
for _, row in df_tendencias.iterrows():
    conn.cursor().execute('''
        INSERT INTO tendencias_colores (id_tendencia, nombre_tendencia, descripcion_tendencia, hex_color, fuente_sitio)
        VALUES (%s, %s, %s, %s, %s)
    ''', (str(uuid.uuid4()), row['nombre_tendencia'], row['descripcion_tendencia'], row['hex_color'], row['fuente_sitio']))

# Insertar catálogo (Ipesa)
for _, row in df_ipesa.iterrows():
    conn.cursor().execute('''
        INSERT INTO catalogo_pinturas (id_color, marca, nombre_color, hex_color)
        VALUES (%s, %s, %s, %s)
    ''', (str(uuid.uuid4()), row['marca'], row['nombre_color'], row['hex_color']))



## 3. Análisis con AI SQL <a name="analisis"></a>

En esta sección se utilizan queries avanzadas en Snowflake para responder preguntas de negocio clave sobre la cobertura y potencial de los colores de Ipesa frente a las tendencias y la competencia.



### 3.1. Query de Cobertura de Tendencias

**Pregunta:** ¿Qué porcentaje de los colores en tendencia para 2025-2026 tiene Ipesa en su catálogo actual?

**Explicación:** Compara los colores de la tabla `tendencias_colores` con los de la tabla `catalogo_pinturas` donde la marca es 'Ipesa', usando una función de similitud de colores.

```sql
-- Query de cobertura de tendencias
WITH tendencia_rgb AS (
  SELECT id_tendencia, nombre_tendencia, hex_color,
    TO_NUMBER(SUBSTR(hex_color, 2, 2), 16) AS r,
    TO_NUMBER(SUBSTR(hex_color, 4, 2), 16) AS g,
    TO_NUMBER(SUBSTR(hex_color, 6, 2), 16) AS b
  FROM tendencias_colores
),
ipesa_rgb AS (
  SELECT id_color, nombre_color, hex_color,
    TO_NUMBER(SUBSTR(hex_color, 2, 2), 16) AS r,
    TO_NUMBER(SUBSTR(hex_color, 4, 2), 16) AS g,
    TO_NUMBER(SUBSTR(hex_color, 6, 2), 16) AS b
  FROM catalogo_pinturas
  WHERE marca = 'Ipesa'
),
matches AS (
  SELECT t.id_tendencia, t.nombre_tendencia, i.id_color, i.nombre_color,
    SQRT(POWER(t.r - i.r, 2) + POWER(t.g - i.g, 2) + POWER(t.b - i.b, 2)) AS color_distance
  FROM tendencia_rgb t
  JOIN ipesa_rgb i
    ON SQRT(POWER(t.r - i.r, 2) + POWER(t.g - i.g, 2) + POWER(t.b - i.b, 2)) < 30
)
SELECT
  COUNT(DISTINCT matches.id_tendencia) * 100.0 / (SELECT COUNT(*) FROM tendencias_colores) AS porcentaje_cubierto
FROM matches;
```



### 3.2. Query de Mejor Match por Marca

**Pregunta:** Para cada color en tendencia, muéstrame el color más cercano que ofrece Ipesa, Comex y Sherwin-Williams.

**Explicación:** Para cada tendencia, encuentra el color más cercano de cada marca.

```sql
-- Query de mejor match por marca
WITH tendencia_rgb AS (
  SELECT id_tendencia, nombre_tendencia, hex_color,
    TO_NUMBER(SUBSTR(hex_color, 2, 2), 16) AS r,
    TO_NUMBER(SUBSTR(hex_color, 4, 2), 16) AS g,
    TO_NUMBER(SUBSTR(hex_color, 6, 2), 16) AS b
  FROM tendencias_colores
),
cat_rgb AS (
  SELECT id_color, marca, nombre_color, hex_color,
    TO_NUMBER(SUBSTR(hex_color, 2, 2), 16) AS r,
    TO_NUMBER(SUBSTR(hex_color, 4, 2), 16) AS g,
    TO_NUMBER(SUBSTR(hex_color, 6, 2), 16) AS b
  FROM catalogo_pinturas
),
matches AS (
  SELECT t.id_tendencia, t.nombre_tendencia, t.hex_color AS hex_tendencia,
         c.marca, c.nombre_color, c.hex_color AS hex_catalogo,
         SQRT(POWER(t.r - c.r, 2) + POWER(t.g - c.g, 2) + POWER(t.b - c.b, 2)) AS color_distance
  FROM tendencia_rgb t
  JOIN cat_rgb c ON 1=1
)
SELECT
  id_tendencia, nombre_tendencia, hex_tendencia,
  FIRST_VALUE(nombre_color) OVER (PARTITION BY id_tendencia, marca ORDER BY color_distance) AS mejor_opcion,
  FIRST_VALUE(hex_catalogo) OVER (PARTITION BY id_tendencia, marca ORDER BY color_distance) AS hex_mejor_opcion,
  marca
FROM matches
QUALIFY ROW_NUMBER() OVER (PARTITION BY id_tendencia, marca ORDER BY color_distance) = 1;
```



### 3.3. Query de Ranking de Potencial de Colores Ipesa

**Pregunta:** Genera un ranking de los 10 colores de Ipesa con mayor potencial de ventas.

**Explicación:** Calcula un score para cada color de Ipesa según su cercanía a las tendencias.

```sql
-- Query de ranking de potencial
WITH tendencia_rgb AS (
  SELECT id_tendencia, hex_color,
    TO_NUMBER(SUBSTR(hex_color, 2, 2), 16) AS r,
    TO_NUMBER(SUBSTR(hex_color, 4, 2), 16) AS g,
    TO_NUMBER(SUBSTR(hex_color, 6, 2), 16) AS b
  FROM tendencias_colores
),
ipesa_rgb AS (
  SELECT id_color, nombre_color, hex_color,
    TO_NUMBER(SUBSTR(hex_color, 2, 2), 16) AS r,
    TO_NUMBER(SUBSTR(hex_color, 4, 2), 16) AS g,
    TO_NUMBER(SUBSTR(hex_color, 6, 2), 16) AS b
  FROM catalogo_pinturas
  WHERE marca = 'Ipesa'
),
scores AS (
  SELECT i.id_color, i.nombre_color, i.hex_color,
    SUM(
      CASE
        WHEN SQRT(POWER(i.r - t.r, 2) + POWER(i.g - t.g, 2) + POWER(i.b - t.b, 2)) < 30
        THEN 1 + (30 - SQRT(POWER(i.r - t.r, 2) + POWER(i.g - t.g, 2) + POWER(i.b - t.b, 2))) / 30
        ELSE 0
      END
    ) AS trend_score
  FROM ipesa_rgb i
  CROSS JOIN tendencia_rgb t
  GROUP BY i.id_color, i.nombre_color, i.hex_color
)
SELECT nombre_color, hex_color, trend_score
FROM scores
ORDER BY trend_score DESC
LIMIT 10;
```



## 4. Visualización de Resultados <a name="visualizacion"></a>

En esta sección se presentan visualizaciones para comunicar los hallazgos del análisis:
- Cobertura de tendencias por marca
- Tabla visual de matches
- Ranking Top 10 colores Ipesa



In [ ]:
# Gráfico de barras: Cobertura de tendencias por marca
import matplotlib.pyplot as plt

marcas = ['Ipesa', 'Comex', 'Sherwin-Williams']
cobertura = [65, 72, 80]  # Reemplaza con los resultados reales

plt.bar(marcas, cobertura, color=['#e74c3c', '#3498db', '#27ae60'])
plt.ylabel('% Cobertura de Tendencias')
plt.title('Cobertura de Colores en Tendencia por Marca')
plt.show()



In [ ]:
# Tabla visual de matches de colores
import plotly.graph_objects as go

# Supón que tienes un DataFrame df_matches con columnas: nombre_tendencia, hex_tendencia, mejor_opcion_ipesa, hex_ipesa, etc.
# df_matches = ...

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_matches.columns)),
    cells=dict(values=[df_matches[col] for col in df_matches.columns],
               fill_color=[['white']*len(df_matches)]*len(df_matches.columns),
               line_color='darkslategray'))
])
fig.show()



In [ ]:
# Ranking Top 10 colores Ipesa para invertir en marketing
# Supón que tienes un DataFrame df_top10 con nombre_color, hex_color, trend_score
# df_top10 = ...

plt.figure(figsize=(10,6))
plt.barh(df_top10['nombre_color'], df_top10['trend_score'], color=df_top10['hex_color'])
plt.xlabel('Trend Score')
plt.title('Top 10 Colores Ipesa para Invertir en Marketing')
plt.gca().invert_yaxis()
plt.show()

